# FBRef

In [40]:
!python -m pip install --upgrade pip
!pip install pandas
!pip install soccerdata
!pip install numpy
!pip install scikit-learn
!pip install matplotlib

  Obtaining dependency information for matplotlib from https://files.pythonhosted.org/packages/4d/9c/65830d4a56c47f5283eaa244dc1228c5da9c844a9f999ebcc2e69bf6cc65/matplotlib-3.7.2-cp311-cp311-win_amd64.whl.metadata
  Using cached matplotlib-3.7.2-cp311-cp311-win_amd64.whl.metadata (5.8 kB)
  Obtaining dependency information for contourpy>=1.0.1 from https://files.pythonhosted.org/packages/16/09/989b982322439faa4bafffcd669e6f942b38fee897c2664c987bcd091dec/contourpy-1.1.0-cp311-cp311-win_amd64.whl.metadata
  Obtaining dependency information for fonttools>=4.22.0 from https://files.pythonhosted.org/packages/52/65/aaa3d2b7a292d93cc2cf1c534d03ba3f744e480f15b3b2ab6ad68189f7ee/fonttools-4.42.0-cp311-cp311-win_amd64.whl.metadata
     ---------------------------------------- 0.0/153.7 kB ? eta -:--:--
     -------------------------------------- 153.7/153.7 kB 9.0 MB/s eta 0:00:00
     ---------------------------------------- 0.0/55.4 kB ? eta -:--:--
     ----------------------------------------

In [41]:
import os
import pandas as pd
import soccerdata as sd
import numpy as np
import sklearn
import matplotlib as plt

sd.FBref.available_leagues()

['Big 5 European Leagues Combined',
 'ENG-Premier League',
 'ESP-La Liga',
 'FRA-Ligue 1',
 'GER-Bundesliga',
 "INT-Women's World Cup",
 'INT-World Cup',
 'ITA-Serie A']

# English Premier League 2022-2023 season

In [9]:
# MUST RUN BEFORE CREATING NEW DATA FILE
fbref = sd.FBref(leagues="ENG-Premier League", seasons=['1819','1920','2021','2122','2223'])
print(fbref.__doc__)

[08/05/23 11:07:48] INFO     Saving cached data to C:\Users\lantz\soccerdata\data\FBref               ]8;id=142245;file://c:\Users\lantz\OneDrive\Documents\GitHub\zen-analytics\venv\Lib\site-packages\soccerdata\_common.py\_common.py]8;;\:]8;id=873820;file://c:\Users\lantz\OneDrive\Documents\GitHub\zen-analytics\venv\Lib\site-packages\soccerdata\_common.py#89\89]8;;\

Provides pd.DataFrames from data at http://fbref.com.

    Data will be downloaded as necessary and cached locally in
    ``~/soccerdata/data/FBref``.

    Parameters
    ----------
    leagues : string or iterable, optional
        IDs of leagues to include. For efficiently reading data from the Top-5
        European leagues, use "Big 5 European Leagues Combined".
    seasons : string, int or list, optional
        Seasons to include. Supports multiple formats.
        Examples: '16-17'; 2016; '2016-17'; [14, 15, 16]
    proxy : 'tor' or dict or list(dict) or callable, optional
        Use a proxy to hide your IP address. Valid options are:
            - "tor": Uses the Tor network. Tor should be running in
              the background on port 9050.
            - dict: A dictionary with the proxy to use. The dict should be
              a mapping of supported protocols to proxy addresses. For example::

                  {
                      'http': 'http://10.10.1.10:3128',
     

c:\Users\lantz\OneDrive\Documents\GitHub\zen-analytics\venv\Lib\site-packages\soccerdata\_common.py:471: UserWarning: Season id "2021" is ambiguous: interpreting as "20-21"
  warnings.warn(msg, stacklevel=1)


## Fetch and clean data

In [17]:
### Get data
stats_pd = fbref.read_player_season_stats(stat_type='standard')

### Convert data types
cols = list(stats_pd.columns)
for col in cols:
    if col in [('nation', ''),('pos', ''),('born', '')]:
        stats_pd[col] = stats_pd[col].astype(str)
    else:
        stats_pd[col] = stats_pd[col].apply(pd.to_numeric)
# print("New_dtypes:")
print(stats_pd.info())

print("\n***** Print data *****")
stats_pd
# print(stats_pd)

<class 'pandas.core.frame.DataFrame'>
MultiIndex: 2677 entries, ('ENG-Premier League', '1819', 'Arsenal', 'Aaron Ramsey') to ('ENG-Premier League', '2223', 'Wolves', 'Toti Gomes')
Data columns (total 33 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   (nation, )                  2677 non-null   object 
 1   (pos, )                     2677 non-null   object 
 2   (age, )                     2677 non-null   int64  
 3   (born, )                    2677 non-null   object 
 4   (Playing Time, MP)          2677 non-null   int64  
 5   (Playing Time, Starts)      2677 non-null   int64  
 6   (Playing Time, Min)         2677 non-null   int64  
 7   (Playing Time, 90s)         2677 non-null   float64
 8   (Performance, Gls)          2677 non-null   int64  
 9   (Performance, Ast)          2677 non-null   int64  
 10  (Performance, G+A)          2677 non-null   int64  
 11  (Performance, G-PK)         2677 non-null

nation    pos age  \
                                                                             
league             season team    player                                     
ENG-Premier League 1819   Arsenal Aaron Ramsey              WAL     MF  27   
                                  Ainsley Maitland-Niles    ENG     DF  20   
                                  Alex Iwobi                NGA  FW,MF  22   
                                  Alexandre Lacazette       FRA     FW  27   
                                  Bernd Leno                GER     GK  26   
...                                                         ...    ...  ..   
                   2223   Wolves  Rayan Aït Nouri           ALG     DF  21   
                                  Raúl Jiménez              MEX     FW  31   
                                  Rúben Neves               POR     MF  25   
                                  Sasa Kalajdzic            AUT     FW  25   
                                  Toti Gomes                POR     DF  23   

                                                          born Playing Time  \
                                                                         MP   
league             season team    player                                      
ENG-Premier League 1819   Arsenal Aaron Ramsey            1990           28   
                                  Ainsley Maitland-Niles  1997           16   
                                  Alex Iwobi              1996           35   
                                  Alexandre Lacazette     1991           35   
                                  Bernd Leno              1992           32   
...                                                        ...          ...   
                   2223   Wolves  Rayan Aït Nouri         2001           21   
                                  Raúl Jiménez            1991           15   
                                  Rúben Neves             1997           35   
                                  Sasa Kalajdzic          1997            1   
                                  Toti Gomes              1999           17   

                                                                             \
                                                         Starts   Min   90s   
league             season team    player                                      
ENG-Premier League 1819   Arsenal Aaron Ramsey               14  1331  14.8   
                                  Ainsley Maitland-Niles     11   987  11.0   
                                  Alex Iwobi                 22  1970  21.9   
                                  Alexandre Lacazette        27  2502  27.8   
                                  Bernd Leno                 31  2836  31.5   
...                                                         ...   ...   ...   
                   2223   Wolves  Rayan Aït Nouri             9  1076  12.0   
                                  Raúl Jiménez                8   846   9.4   
                                  Rúben Neves                33  3019  33.5   
                                  Sasa Kalajdzic              1    45   0.5   
                                  Toti Gomes                 10   984  10.9   

                                                         Performance      ...  \
                                                                 Gls Ast  ...   
league             season team    player                                  ...   
ENG-Premier League 1819   Arsenal Aaron Ramsey                     4   6  ...   
                                  Ainsley Maitland-Niles           1   1  ...   
                                  Alex Iwobi                       3   6  ...   
                                  Alexandre Lacazette             13   8  ...   
                                  Bernd Leno                       0   0  ...   
...                                                              ...  ..  ...   
                   22

### Data Analysis

In [29]:
stats_pd_data = stats_pd.drop(columns=[('nation', ''),('pos', ''),('born', '')])
stats_pd_data.head()

age Playing Time  \
                                                                       MP   
league             season team    player                                    
ENG-Premier League 1819   Arsenal Aaron Ramsey            27           28   
                                  Ainsley Maitland-Niles  20           16   
                                  Alex Iwobi              22           35   
                                  Alexandre Lacazette     27           35   
                                  Bernd Leno              26           32   

                                                                             \
                                                         Starts   Min   90s   
league             season team    player                                      
ENG-Premier League 1819   Arsenal Aaron Ramsey               14  1331  14.8   
                                  Ainsley Maitland-Niles     11   987  11.0   
                                  Alex Iwobi                 22  1970  21.9   
                                  Alexandre Lacazette        27  2502  27.8   
                                  Bernd Leno                 31  2836  31.5   

                                                         Performance          \
                                                                 Gls Ast G+A   
league             season team    player                                       
ENG-Premier League 1819   Arsenal Aaron Ramsey                     4   6  10   
                                  Ainsley Maitland-Niles           1   1   2   
                                  Alex Iwobi                       3   6   9   
                                  Alexandre Lacazette             13   8  21   
                                  Bernd Leno                       0   0   0   

                                                                  ...  \
                                                         G-PK PK  ...   
league             season team    player                          ...   
ENG-Premier League 1819   Arsenal Aaron Ramsey              4  0  ...   
                                  Ainsley Maitland-Niles    1  0  ...   
                                  Alex Iwobi                3  0  ...   
                                  Alexandre Lacazette      13  0  ...   
                                  Bernd Leno                0  0  ...   

                                                         Per 90 Minutes        \
                                                                    Gls   Ast   
league             season team    player                                        
ENG-Premier League 1819   Arsenal Aaron Ramsey                     0.27  0.41   
                                  Ainsley Maitland-Niles           0.09  0.09   
                                  Alex Iwobi                       0.14  0.27   
                                  Alexandre Lacazette              0.47  0.29   
                                  Bernd Leno                       0.00  0.00   

                                                                             \
                                                           G+A  G-PK G+A-PK   
league             season team    player                                      
ENG-Premier League 1819   Arsenal Aaron Ramsey            0.68  0.27   0.68   
                                  Ainsley Maitland-Niles  0.18  0.09   0.18   
                                  Alex Iwobi              0.41  0.14   0.41   
                                  Alexandre Lacazette     0.76  0.47   0.76   
                                  Bernd Leno              0.00  0.00   0.00   

                                                                             \
                                                            xG   xAG xG+xAG   
league             season team    player                                      
ENG-Premier League 1819   Arsenal Aaron Ramsey            0.23

In [30]:
len(stats_pd_data)

2677

#### Get statistical overview of all player stats

In [31]:
stats_pd_data.describe()

age Playing Time                                         \
                             MP       Starts          Min          90s   
count  2677.000000  2677.000000  2677.000000  2677.000000  2677.000000   
mean     25.521106    19.916698    15.614494  1402.814718    15.586291   
std       4.315217    11.770228    11.932768  1036.947125    11.521911   
min      15.000000     1.000000     0.000000     1.000000     0.000000   
25%      22.000000     9.000000     4.000000   445.000000     4.900000   
50%      25.000000    21.000000    14.000000  1284.000000    14.300000   
75%      29.000000    30.000000    26.000000  2253.000000    25.000000   
max      39.000000    38.000000    38.000000  3420.000000    38.000000   

       Performance                                                      ...  \
               Gls          Ast          G+A         G-PK           PK  ...   
count  2677.000000  2677.000000  2677.000000  2677.000000  2677.000000  ...   
mean      1.906612     1.345162     3.251774     1.751214     0.155398  ...   
std       3.472541     2.134585     5.000535     3.109265     0.722103  ...   
min       0.000000     0.000000     0.000000     0.000000     0.000000  ...   
25%       0.000000     0.000000     0.000000     0.000000     0.000000  ...   
50%       1.000000     0.000000     1.000000     1.000000     0.000000  ...   
75%       2.000000     2.000000     4.000000     2.000000     0.000000  ...   
max      36.000000    20.000000    44.000000    29.000000    10.000000  ...   

      Per 90 Minutes                                                      \
                 Gls          Ast          G+A         G-PK       G+A-PK   
count    2677.000000  2677.000000  2677.000000  2677.000000  2677.000000   
mean        0.105992     0.086843     0.192761     0.099369     0.186130   
std         0.177114     0.426770     0.472419     0.168326     0.468222   
min         0.000000     0.000000     0.000000     0.000000     0.000000   
25%         0.000000     0.000000     0.000000     0.000000     0.000000   
50%         0.030000     0.000000     0.090000     0.030000     0.090000   
75%         0.140000     0.120000     0.280000     0.130000     0.270000   
max         2.030000    18.000000    18.000000     2.030000    18.000000   

                                                                        
                xG          xAG       xG+xAG         npxG     npxG+xAG  
count  2668.000000  2668.000000  2668.000000  2668.000000  2668.000000  
mean      0.131855     0.088137     0.219963     0.125056     0.213201  
std       0.207641     0.258677     0.346287     0.201056     0.341304  
min       0.000000     0.000000     0.000000     0.000000     0.000000  
25%       0.020000     0.010000     0.050000     0.020000     0.050000  
50%       0.060000     0.060000     0.130000     0.060000     0.130000  
75%       0.170000     0.120000     0.310000     0.170000     0.300000  
max       4.500000     9.730000     9.730000     4.500000     9.730000  

[8 rows x 30 columns]

In [32]:
from sklearn.feature_selection import VarianceThreshold
sel = VarianceThreshold(threshold=(.8 * (1 - .8)))
sel.fit_transform(stats_pd_data)

array([[2.7e+01, 2.8e+01, 1.4e+01, ..., 4.1e-01, 6.8e-01, 6.8e-01],
       [2.0e+01, 1.6e+01, 1.1e+01, ..., 9.0e-02, 1.8e-01, 1.8e-01],
       [2.2e+01, 3.5e+01, 2.2e+01, ..., 2.7e-01, 4.1e-01, 4.1e-01],
       ...,
       [2.5e+01, 3.5e+01, 3.3e+01, ..., 3.0e-02, 2.1e-01, 1.2e-01],
       [2.5e+01, 1.0e+00, 1.0e+00, ..., 0.0e+00, 0.0e+00, 0.0e+00],
       [2.3e+01, 1.7e+01, 1.0e+01, ..., 0.0e+00, 9.0e-02, 9.0e-02]])

#### Create data correlation matrix

In [37]:
matrix = stats_pd_data.corr()
print("Correlation matrix is : ")
print(matrix)

Correlation matrix is : 
                              age Playing Time                                \
                                            MP    Starts       Min       90s   
age                      1.000000     0.170172  0.190972  0.193253  0.193234   
Playing Time   MP        0.170172     1.000000  0.926151  0.938308  0.938319   
               Starts    0.190972     0.926151  1.000000  0.996353  0.996345   
               Min       0.193253     0.938308  0.996353  1.000000  0.999997   
               90s       0.193234     0.938319  0.996345  0.999997  1.000000   
Performance    Gls       0.039908     0.453496  0.412377  0.408504  0.408518   
               Ast       0.022698     0.497952  0.458633  0.455067  0.455094   
               G+A       0.037403     0.527485  0.482146  0.477934  0.477956   
               G-PK      0.027826     0.456849  0.409973  0.405714  0.405730   
               PK        0.072102     0.213710  0.217812  0.217525  0.217523   
               

#### Get statistical overview of all player stats grouped by player

In [52]:
playerGroups = stats_pd_data.groupby('player')
playerGroups.describe()

age                                                       \
                                                                              
                 count       mean       std   min    25%   50%    75%   max   
player                                                                        
Aaron Connolly     3.0  20.000000  1.000000  19.0  19.50  20.0  20.50  21.0   
Aaron Cresswell    5.0  30.000000  1.581139  28.0  29.00  30.0  31.00  32.0   
Aaron Hickey       1.0  20.000000       NaN  20.0  20.00  20.0  20.00  20.0   
Aaron Lennon       3.0  32.333333  1.527525  31.0  31.50  32.0  33.00  34.0   
Aaron Mooy         2.0  27.500000  0.707107  27.0  27.25  27.5  27.75  28.0   
...                ...        ...       ...   ...    ...   ...    ...   ...   
Érik Lamela        3.0  27.000000  1.000000  26.0  26.50  27.0  27.50  28.0   
Étienne Capoue     2.0  30.500000  0.707107  30.0  30.25  30.5  30.75  31.0   
Ørjan Nyland       1.0  28.000000       NaN  28.0  28.00  28.0  28.00  28.0   
İlkay Gündoğan     5.0  29.000000  1.581139  27.0  28.00  29.0  30.00  31.0   
Łukasz Fabiański   5.0  35.000000  1.581139  33.0  34.00  35.0  36.00  37.0   

                 Playing Time             ... Per 90 Minutes                 \
                           MP             ...           npxG       npxG+xAG   
                        count       mean  ...            75%   max    count   
player                                    ...                                 
Aaron Connolly            3.0  15.000000  ...         0.3650  0.45      3.0   
Aaron Cresswell           5.0  29.200000  ...         0.0400  0.05      5.0   
Aaron Hickey              1.0  26.000000  ...         0.0200  0.02      1.0   
Aaron Lennon              3.0  20.000000  ...         0.0600  0.06      3.0   
Aaron Mooy                2.0  30.000000  ...         0.0950  0.10      2.0   
...                       ...        ...  ...            ...   ...      ...   
Érik Lamela               3.0  22.333333  ...         0.2650  0.31      3.0   
Étienne Capoue            2.0  31.500000  ...         0.0375  0.04      2.0   
Ørjan Nyland              1.0   7.000000  ...         0.0000  0.00      1.0   
İlkay Gündoğan            5.0  29.600000  ...         0.3300  0.44      5.0   
Łukasz Fabiański          5.0  34.200000  ...         0.0000  0.00      5.0   

                                                                         
                                                                         
                      mean       std   min     25%    50%     75%   max  
player                                                                   
Aaron Connolly    0.353333  0.113725  0.26  0.2900  0.320  0.4000  0.48  
Aaron Cresswell   0.150000  0.043012  0.09  0.1400  0.150  0.1600  0.21  
Aaron Hickey      0.040000       NaN  0.04  0.0400  0.040  0.0400  0.04  
Aaron Lennon      0.076667  0.041633  0.03  0.0600  0.090  0.1000  0.11  
Aaron Mooy        0.235000  0.077782  0.18  0.2075  0.235  0.2625  0.29  
...                    ...       ...   ...     ...    ...     ...   ...  
Érik Lamela       0.346667  0.117189  0.26  0.2800  0.300  0.3900  0.48  
Étienne Capoue    0.105000  0.049497  0.07  0.0875  0.105  0.1225  0.14  
Ørjan Nyland      0.000000       NaN  0.00  0.0000  0.000  0.0000  0.00  
İlkay Gündoğan    0.416000  0.173868  0.20  0.3500  0.380  0.4800  0.67  
Łukasz Fabiański  0.000000  0.000000  0.00  0.0000  0.000  0.0000  0.00  

[1156 rows x 240 columns]

#### Get statistical overview of player goals grouped by player

In [53]:
playerGroups.describe()['Performance']['Gls']

,count,mean,std,min,25%,50%,75%,max
player,,,,,,,,
Aaron Connolly,3.0,1.666667,1.527525,0.0,1.00,2.0,2.50,3.0
Aaron Cresswell,5.0,1.000000,1.414214,0.0,0.00,0.0,2.00,3.0
Aaron Hickey,1.0,0.000000,NaN,0.0,0.00,0.0,0.00,0.0
Aaron Lennon,3.0,1.000000,1.000000,0.0,0.50,1.0,1.50,2.0
Aaron Mooy,2.0,2.500000,0.707107,2.0,2.25,2.5,2.75,3.0
...,...,...,...,...,...,...,...,...
Érik Lamela,3.0,2.333333,1.527525,1.0,1.50,2.0,3.00,4.0
Étienne Capoue,2.0,0.500000,0.707107,0.0,0.25,0.5,0.75,1.0
Ørjan Nyland,1.0,0.000000,NaN,0.0,0.00,0.0,0.00,0.0


#### Get the average number of player goals grouped by player

In [57]:
top_25 = playerGroups.mean()['Performance']['Gls'].sort_values(ascending=False).head(25)
top_25

player
Erling Haaland               36.000000
Harry Kane                   21.000000
Mohamed Salah                21.000000
Sadio Mané                   16.750000
Ivan Toney                   16.000000
Eden Hazard                  16.000000
Jamie Vardy                  14.800000
Son Heung-min                14.600000
Sergio Agüero                13.666667
Ollie Watkins                13.333333
Raheem Sterling              13.200000
Callum Wilson                12.000000
Marcus Rashford              11.800000
Pierre-Emerick Aubameyang    11.800000
Teemu Pukki                  11.000000
Gareth Bale                  11.000000
Bruno Fernandes              11.000000
Matheus Pereira              11.000000
Tammy Abraham                10.500000
Marko Arnautović             10.000000
Romelu Lukaku                10.000000
Alexandre Lacazette          10.000000
Alexander Isak               10.000000
Taiwo Awoniyi                10.000000
Gabriel Jesus                 9.800000
Name: Gls, dtype: 